# Detention By Nationality By Year Analysis

For details about the analysis below, please [read this page](https://github.com/BuzzFeedNews/2016-01-immigrant-detention-india).

## Load the data

In [1]:
import pandas as pd
import sys
sys.path.append("../utils")
import loaders

*Note: loaders is a custom module to handle basic data-loading. It is available [here](https://github.com/BuzzFeedNews/2015-08-immigration/blob/master/utils/loaders.py).*

In [2]:
first_scheduled_proceeding = pd.read_csv("../data/first_scheduled_proceeding.csv", 
    parse_dates=["ADJ_DATE"],
    dtype={ "IDNCASE": str, "IDNPROCEEDING": str },
    encoding="latin-1")

*Note: first-scheduled-proceeding.csv is a pre-processed data file. The code to create that file from tbl_schedule.csv is available [here](../utils/generate-first-scheduled-proceeding.py).*

In [3]:
nationality_table = loaders.load_file("../data/tblLookupNationality.csv")

In [4]:
case_date_list = [
    "E_28_DATE",
    "DATE_OF_ENTRY",
    "C_BIRTHDATE",
    "C_RELEASE_DATE",
    "DATE_DETAINED",
    "DATE_RELEASED"
]

In [5]:
_cases = loaders.load_file("../data/A_tblCase.csv",
    parse_dates=case_date_list,
    dtype={ "IDNCASE": str })

In [6]:
_cases["GENDER"] = _cases["GENDER"].fillna("UNK")

In [7]:
_charges = loaders.load_file("../data/B_tblProceedCharges.csv",
    dtype={ "IDNCASE": str, "IDNPROCEEDING": str })

Skipping line 1165848: expected 5 fields, saw 6

Skipping line 1433634: expected 5 fields, saw 6

Skipping line 2646392: expected 5 fields, saw 6

Skipping line 2847501: expected 5 fields, saw 6

Skipping line 2947399: expected 5 fields, saw 6

Skipping line 3131015: expected 5 fields, saw 6



*Note: Six rows — of the more than 8 million total rows — in the charges table contain malformed data stemming from extra tab characters, triggering the warning messages above.*

## Process the data

Join the various tables and prepare them for analysis.

In [8]:
charges_group = _charges.groupby([ "IDNCASE", "IDNPROCEEDING" ])

In [9]:
charge_lists = pd.DataFrame({
    "charge_list": charges_group["CHARGE"].apply("|".join)
}).reset_index()

In [10]:
charge_lists.head()

,IDNCASE,IDNPROCEEDING,charge_list
0,2046920,3200048,212a06Ai
1,2046921,3200049,212a06Ai
2,2046922,3200050,212a06Ai
3,2046923,3200051,212a06Ci
4,2046923,3525150,212a06Ci


In [11]:
assert(charge_lists["IDNCASE"].nunique() == 5033293)
assert(len(first_scheduled_proceeding) == 5045511)

From the numbers above: A small fraction of cases — approximately 0.2% — have a scheduled proceding but no charges.

In [12]:
cases_with_first_proceeding = first_scheduled_proceeding\
    .merge(charge_lists, how="left", on=[ "IDNCASE", "IDNPROCEEDING" ])\
    .merge(_cases, how="left", on="IDNCASE", suffixes=["_schedule", "_case"])

Legal representatives file the EOIR-28 form to notify the court of their representation for a given immigrant.

`ADJ_DATE` in this table indicates the date of the case's first proceeding.

In [13]:
cases_with_first_proceeding["legal_rep_at_first_proceeding"] = \
    cases_with_first_proceeding["E_28_DATE"] <= cases_with_first_proceeding["ADJ_DATE"]

## Select non-criminal removal cases between Jan. 1, 2003 and Jan. 1, 2015

In [14]:
selected_cases = cases_with_first_proceeding[
     # Select cases with first-scheduled-hearing dates in 2003–2014
    (cases_with_first_proceeding["ADJ_DATE"] >= "2003-01-01") &
    (cases_with_first_proceeding["ADJ_DATE"] < "2015-01-01") &
    # Remove unaccompanied children
    (cases_with_first_proceeding["CASEPRIORITY_CODE"] != "UC") & 
    # Keep only "removal" cases
    (cases_with_first_proceeding["CASE_TYPE"] == "RMV")
].copy()

In [15]:
selected_cases["has_criminal_charge"] = (
    selected_cases["charge_list"].str.contains("237a02") |
    selected_cases["charge_list"].str.contains("212a02")
)

In [16]:
selected_cases["detained"] = selected_cases["CUSTODY"].map({"N": 0, "D": 1, "R": 1})

In [17]:
non_crim_selected_cases = selected_cases[~selected_cases["has_criminal_charge"]].copy()

## Calculate detention rates by nationality by year

In [18]:
non_crim_selected_cases["year"] = non_crim_selected_cases["ADJ_DATE"].apply(lambda x: x.year)

In [19]:
non_crim_custody_by_nationality = non_crim_selected_cases.groupby(["year", "NAT", "CUSTODY"])\
    .size()\
    .unstack()\
    .fillna(0)

In [20]:
non_crim_custody_by_nationality["total"] = non_crim_custody_by_nationality.sum(axis=1)

In [21]:
non_crim_custody_by_nationality["percent_detained"] = non_crim_custody_by_nationality\
    .apply(lambda x: round(100.0 * (x["D"] + x["R"]) / x["total"], 1), axis=1)

In [22]:
non_crim_custody_by_nationality.tail()

CUSTODY   D   N  R  total  percent_detained
year NAT                                   
2014 YE   5  32  7     44              27.3
     YO   0  38  3     41               7.3
     YS   4  19  6     29              34.5
     ZA   7  14  3     24              41.7
     ZI   7  28  5     40              30.0

In [23]:
#Add full country names
non_crim_custody_by_nationality_by_year = non_crim_custody_by_nationality.reset_index().set_index("NAT")\
    .join(nationality_table.set_index("NAT_CODE")[[ "NAT_NAME" ]])
non_crim_custody_by_nationality_by_year.tail()

CUSTODY,year,D,N,R,total,percent_detained,NAT_NAME
ZI,2010,15,137,16,168,18.5,ZIMBABWE
ZI,2011,13,86,7,106,18.9,ZIMBABWE
ZI,2012,9,78,9,96,18.8,ZIMBABWE
ZI,2013,7,38,6,51,25.5,ZIMBABWE
ZI,2014,7,28,5,40,30.0,ZIMBABWE


In [24]:
main_columns = ["N", "D", "R", "total", "percent_detained", "year", "NAT_NAME"]

large_nationalities = non_crim_custody_by_nationality_by_year[
    non_crim_custody_by_nationality_by_year["total"] > 1000
].sort_values("percent_detained", ascending=False)[main_columns]

large_nationalities.head()

CUSTODY,N,D,R,total,percent_detained,year,NAT_NAME
IN,692,147,3341,4180,83.4,2013,INDIA
MX,14983,60468,8247,83698,82.1,2008,MEXICO
MX,12898,46162,5954,65014,80.2,2007,MEXICO
GT,3498,8416,5045,16959,79.4,2011,GUATEMALA
MX,11924,42267,2259,56450,78.9,2005,MEXICO


## Average Non-Criminal Detention Rate By Year

In [25]:
non_crim_year_totals = non_crim_selected_cases.groupby(["year", "CUSTODY"])\
    .size()\
    .unstack()\
    .fillna(0)

In [26]:
non_crim_year_totals["total"] = non_crim_year_totals.sum(axis=1)

In [27]:
non_crim_year_totals["percent_detained"] = non_crim_year_totals\
    .apply(lambda x: round(100.0 * (x["D"] + x["R"]) / x["total"], 1), axis=1)

In [28]:
non_crim_year_totals.ix[2009:2014]

CUSTODY,D,N,R,total,percent_detained
year,,,,,
2009,89500,81982,25761,197243,58.4
2010,76001,82604,35250,193855,57.4
2011,78508,72375,47132,198015,63.4
2012,54683,62371,51562,168616,63.0
2013,34773,62300,56707,153780,59.5
2014,33878,74184,63889,171951,56.9


## Indian Immigrant Detention Rate By Year

In [29]:
large_nationalities[
    (large_nationalities["NAT_NAME"] == "INDIA")
][["NAT_NAME", "year", "N", "D", "R", "total", "percent_detained"]]\
    .sort_values("year")\
    .set_index("year").ix[2009:2014]

CUSTODY,NAT_NAME,N,D,R,total,percent_detained
year,,,,,,
2009,INDIA,1666,131,265,2062,19.2
2010,INDIA,1556,157,1526,3239,52.0
2011,INDIA,1456,217,2160,3833,62.0
2012,INDIA,933,98,717,1748,46.6
2013,INDIA,692,147,3341,4180,83.4
2014,INDIA,648,151,1698,2497,74.0


----

----

----